In [1]:
print('Materialisation Data Test')

Materialisation Data Test


In [2]:
import os
import compas
from compas.datastructures import Mesh, mesh_bounding_box_xy
from compas.geometry import Vector, Frame, Scale

HERE = os.getcwd()

FILE_I = os.path.join(HERE, 'blocks and ribs_RHINO', 'sessions', 'bm_vertical_equilibrium', 'simple_tripod.rv2')
FILE_O1 = os.path.join(HERE, 'blocks and ribs_RHINO', 'data', 'form.json')
FILE_O2 = os.path.join(HERE, 'blocks and ribs_RHINO', 'data', 'scaled_form.json')

session = compas.json_load(FILE_I)

mesh = Mesh.from_data(session['data']['form'])

### to delete extra faces(more than 4 edges) if subdivided with catmulclark or other weird subdivision that connects the mesh with the ground

In [3]:
delete_faces =[]

for fkey in mesh.faces():
    if len(mesh.face_vertices(fkey)) > 4:
        delete_faces.append(fkey)

for fkey in delete_faces:       
    mesh.delete_face(fkey)
    mesh.remove_unused_vertices()

### scale up the form if needed

In [4]:
scaled_mesh = mesh.copy()

box_points = mesh_bounding_box_xy(scaled_mesh)
base_mesh = scaled_mesh.from_points(box_points)
centroid = base_mesh.centroid()
#print (centroid)
frame = Frame(centroid,Vector(1,0,0),Vector(0,1,0))

S = Scale.from_factors([100, 100, 100], frame)
scaled_mesh.transform(S)

### Visualise and export Initial Mesh

In [5]:
mesh.to_json(FILE_O1)
scaled_mesh.to_json(FILE_O2)

print(mesh)

<Mesh with 37 vertices, 24 faces, 60 edges>


In [6]:
from pythreejs import *
import numpy as np
from IPython.display import display

vertices = []
for face in mesh.faces():
    for v in mesh.face_vertices(face):
        xyz = mesh.vertex_attributes(v, "xyz")
        vertices.append(xyz)

print(vertices)

[[-4.223588094538546, -2.049343512703559, 11.75487914807182], [3.3168890776086184, -0.29503424492501495, 14.727819542584887], [-4.305358833080639, 8.309585822394906, 10.17640364037847], [-7.479312274451397, 6.859611040439988, 8.334298209825157], [-4.305358833080639, 8.309585822394906, 10.17640364037847], [-10.705645161290324, 15.92741935483871, 0.0], [-11.9007168252293, 15.10626581888064, 0.0], [-7.479312274451397, 6.859611040439988, 8.334298209825157], [-11.9007168252293, 15.10626581888064, 0.0], [-13.095788489168275, 14.285112282922572, 0.0], [-9.487585344957099, 6.071936033187963, 5.364442791079558], [-7.479312274451397, 6.859611040439988, 8.334298209825157], [-9.487585344957099, 6.071936033187963, 5.364442791079558], [-7.348177668464897, -2.7173239661514548, 7.5029938028193035], [-4.223588094538546, -2.049343512703559, 11.75487914807182], [-7.479312274451397, 6.859611040439988, 8.334298209825157], [5.99353911452859, 6.19208075151476, 11.503994506875527], [3.3168890776086184, -0.295

In [7]:
vertices = BufferAttribute(
    array = np.array(vertices,dtype=np.float32),
    normalized = False)

geometry = BufferGeometry(
    attribute={'position': vertices})

geometry.exec_three_obj_method('computeVertexNormals')

mesh_3j = Mesh(geometry=geometry,
           material=MeshPhongMaterial(color='#0092D2'),
           position=[0,0,0])

In [8]:
c = PerspectiveCamera(position = [0, 5, 5], up = [0, 1, 0],
                     children=[DirectionalLight(color='white', position=[3,5,1], intensity=0.5)])

scene=Scene(children=[mesh_3j,c, AmbientLight(color='#777777')])

renderer = Renderer(camera=c, scene=scene, controls=[OrbitControls(controlling=c)],
                   width=800, height=600)
display(renderer)

print(geometry)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…

BufferGeometry()


In [10]:
from pythreejs._example_helper import use_example_model_ids
use_example_model_ids()
BoxGeometry(
    width=5,
    height=10,
    depth=15,
    widthSegments=5,
    heightSegments=10,
    depthSegments=15)

Preview(child=BoxGeometry(depth=15.0, depthSegments=15, height=10.0, heightSegments=10, width=5.0, widthSegmen…